In [1]:
import argparse
import re
import os, glob, datetime
import numpy as np
from keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract
from keras.models import Model, load_model
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import Adam
import data_generator as dg
import keras.backend as K

Using TensorFlow backend.


In [2]:
import sys
sys.path.append("./Func")
from editing_network import NetworkEdit_base

In [3]:
def DnCNN(depth,filters=64,image_channels=1, use_bnorm=True):
    layer_count = 0
    inpt = Input(shape=(None,None,image_channels),name = 'input'+str(layer_count))
    # 1st layer, Conv+relu
    layer_count += 1
    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',name = 'conv'+str(layer_count))(inpt)
    layer_count += 1
    x = Activation('relu',name = 'relu'+str(layer_count))(x)
    # depth-2 layers, Conv+BN+relu
    for i in range(depth-2):
        layer_count += 1
        x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)
        if use_bnorm:
            layer_count += 1
            #x = BatchNormalization(axis=3, momentum=0.1,epsilon=0.0001, name = 'bn'+str(layer_count))(x) 
        x = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(layer_count))(x)
        layer_count += 1
        x = Activation('relu',name = 'relu'+str(layer_count))(x)  
    # last layer, Conv
    layer_count += 1
    x = Conv2D(filters=image_channels, kernel_size=(3,3), strides=(1,1), kernel_initializer='Orthogonal',padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)
    layer_count += 1
    x = Subtract(name = 'subtract' + str(layer_count))([inpt, x])   # input - noise
    model = Model(inputs=inpt, outputs=x)
    
    return model

In [4]:
def lr_schedule(epoch):
    initial_lr = 1e-3
    if epoch<=30:
        lr = initial_lr
    elif epoch<=60:
        lr = initial_lr/10
    elif epoch<=80:
        lr = initial_lr/20 
    else:
        lr = initial_lr/20 
    print('current learning rate is %2.8f' %lr)
    return lr

In [5]:
def train_datagen(epoch_iter=2000,epoch_num=5,batch_size=128,data_dir="/data/BSDS300/images/train/"):
    while(True):
        n_count = 0
        if n_count == 0:
            #print(n_count)
            xs = dg.datagenerator(data_dir)
            assert len(xs)%batch_size ==0, \
            log('make sure the last iteration has a full batchsize, this is important if you use batch normalization!')
            xs = xs.astype('float32')/255.0
            indices = list(range(xs.shape[0]))
            n_count = 1
        for _ in range(epoch_num):
            np.random.shuffle(indices)    # shuffle
            for i in range(0, len(indices), batch_size):
                batch_x = xs[indices[i:i+batch_size]]
                noise =  np.random.normal(0, np.random.rand()*50/255.0, batch_x.shape)    # noise
                #noise =  K.random_normal(ge_batch_y.shape, mean=0, stddev=args.sigma/255.0)
                batch_y = batch_x + noise 
                yield batch_y, batch_x

In [6]:
# data_dir="/data/BSDS300/images/train/"
# xg = dg.datagenerator(data_dir)

In [7]:
# print(xg.shape)

In [8]:
def sum_squared_error(y_true, y_pred):
    #return K.mean(K.square(y_pred - y_true), axis=-1)
    #return K.sum(K.square(y_pred - y_true), axis=-1)/2
    return K.sum(K.square(y_pred - y_true))/2

In [9]:
model = DnCNN(depth=17,filters=64,image_channels=1,use_bnorm=True)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input0 (InputLayer)             (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 640         input0[0][0]                     
__________________________________________________________________________________________________
relu2 (Activation)              (None, None, None, 6 0           conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, None, None, 6 36864       relu2[0][0]                      
__________________________________________________________________________________________________
bn4 (Batch

In [ ]:
model.compile(optimizer=Adam(0.001), loss=sum_squared_error)
lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
history = model.fit_generator(train_datagen(batch_size=128),
            steps_per_epoch=2000, epochs=100, verbose=1, initial_epoch=0,
            callbacks=[lr_scheduler])

current learning rate is 0.00100000
Epoch 1/100
^_^-training data finished-^_^
1363/2000 [===================>..........] - ETA: 1:53 - loss: 791.7433

In [ ]:
save_path = "./saved_model/"
model_name = "fullmodel.h5"
model_architecture = "model_architecture.json"
model_weights = "model_weights.h5"
my_model = NetworkEdit_base(model=model)
my_model.SaveModel(save_content="MODEL", save_path=save_path, save_filename=model_name)
my_model.SaveModel(save_content="ARCHITECTURE", save_path=save_path, save_filename=model_architecture)
my_model.SaveModel(save_content="WEIGHTS", save_path=save_path, save_filename=model_weights)